# Normalizacion de texto

La normalización de texto se define como un proceso que consiste en una serie de pasos que se deben seguir para limpiar y estandarizar los datos textuales en una forma que puedan ser consumidos por otros sistemas y aplicaciones de NLP como entrada.

A menudo, la tokenización en sí también es parte de la normalización del texto. Además de la tokenización, varias otras técnicas incluyen la limpieza del texto, la conversión de mayúsculas y minúsculas, la corrección ortográfica, la eliminación de palabras vacías y otros términos innecesarios, la derivación y la lematización. 


In [1]:
import nltk
import re
import string
from pprint import pprint
corpus = ["The brown fox wasn't that quick and he couldn't win the race",
          "Hey that's a great deal! I just bought a phone for $199",
          "@@You'll (learn) a **lot** in the book. Python is an amazing language !@@"]

A menudo, los datos textuales que queremos usar o analizar contienen muchos tokens y caracteres extraños e innecesarios que deben eliminarse antes de realizar otras operaciones como tokenización u otras técnicas de normalización. Esto incluye extraer texto significativo de fuentes de datos como datos HTML o incluso datos de fuentes XML y JSON.

Hay muchas formas de analizar y limpiar estos datos para eliminar etiquetas innecesarias. Puedes usar funciones como clean_html() de nltk o incluso la biblioteca BeautifulSoup para analizar datos HTML. También puedes usar su propia lógica personalizada, incluidas expresiones regulares, xpath y la biblioteca lxml, para analizar datos XML.

## 1. Tokenizacion
Por lo general, tokenizamos el texto antes o después de eliminar caracteres y símbolos innecesarios de los datos. Esta elección depende del problema que está tratando de resolver y de los datos con los que está tratando. Ya hemos visto varias técnicas de tokenización en el video anterior. Definiremos una función de tokenización genérica aquí y ejecutaremos lo mismo en nuestro corpus mencionado anteriormente.

In [2]:
def tokenize_text(text):
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentence) for sentence in sentences]
    return word_tokens

In [3]:
token_list = [tokenize_text(text) for text in corpus]
print(token_list)

[[['The', 'brown', 'fox', 'was', "n't", 'that', 'quick', 'and', 'he', 'could', "n't", 'win', 'the', 'race']], [['Hey', 'that', "'s", 'a', 'great', 'deal', '!'], ['I', 'just', 'bought', 'a', 'phone', 'for', '$', '199']], [['@', '@', 'You', "'ll", '(', 'learn', ')', 'a', '*', '*', 'lot', '*', '*', 'in', 'the', 'book', '.'], ['Python', 'is', 'an', 'amazing', 'language', '!'], ['@', '@']]]


## 2. Remocion de carcateres especiales

Una tarea importante en la normalización de texto consiste en eliminar caracteres innecesarios y especiales. Estos pueden ser símbolos especiales o incluso signos de puntuación que aparecen en las oraciones.

Este paso a menudo se realiza antes o después de la tokenización. La razón principal para hacerlo es que, a menudo, la puntuación o los caracteres especiales no tienen mucha importancia cuando analizamos el texto y lo utilizamos para extraer características o información basada en NLP y ML.

Implementaremos ambos tipos de eliminación de caracteres especiales, antes y después de la tokenización.

In [4]:
def remove_characters_after_tokenization(tokens):
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
    return filtered_tokens

In [7]:
# Funcion para limpieza antes de tokenizacion
filtered_list_1 = [filter(None,[remove_characters_after_tokenization(tokens) \
                                for tokens in sentence_tokens]) \
                   for sentence_tokens in token_list]
print(filtered_list_1)

[<filter object at 0x000002281AAA03A0>, <filter object at 0x000002281AAA0AC0>, <filter object at 0x000002281AAA0640>]


Esencialmente, lo que hacemos aquí es usar el atributo `string.punctuation`, que consta de todos los caracteres/símbolos especiales posibles, y crear un patrón de expresiones regulares a partir de él. Lo usamos para unir tokens que son símbolos y caracteres y eliminarlos. La función de filter nos ayuda a eliminar los tokens vacíos obtenidos después de eliminar los tokens de caracteres especiales usando el submétodo regex

In [8]:
def remove_characters_before_tokenization(sentence, keep_apostrophes=False):
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]' # add other characters here to remove them
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    else:
        PATTERN = r'[^a-zA-Z0-9 ]' # only extract alpha-numeric characters
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    return filtered_sentence

In [9]:
filtered_list_2 = [remove_characters_before_tokenization(sentence) for sentence in corpus]
print(filtered_list_2)

['The brown fox wasnt that quick and he couldnt win the race', 'Hey thats a great deal I just bought a phone for 199', 'Youll learn a lot in the book Python is an amazing language ']


In [10]:
cleaned_corpus = [remove_characters_before_tokenization(sentence,keep_apostrophes=True)\
                  for sentence in corpus]
cleaned_corpus

["The brown fox wasn't that quick and he couldn't win the race",
 "Hey that's a great deal! I just bought a phone for 199",
 "You'll learn a lot in the book. Python is an amazing language !"]

Los resultados anteriores muestran dos formas diferentes de eliminar caracteres especiales antes de la tokenización: eliminar todos los caracteres especiales en lugar de conservar los apóstrofes utilizando expresiones regulares. Por lo general, después de eliminar estos caracteres, puede tomar el texto limpio y tokenizarlo o aplicarle otras operaciones de normalización.

## 3. Expansion de Contracciones

Las contracciones son versiones abreviadas de palabras o sílabas. Existen en formas escritas o habladas. Se crean versiones abreviadas de palabras existentes eliminando letras y sonidos específicos. En el caso de las contracciones en inglés, a menudo se crean eliminando una de las vocales de la palabra. Los ejemplos serían is not to is not y will not to will not , donde puede notar que el apóstrofe se usa para denotar la contracción y algunas de las vocales y otras letras se eliminan. Por lo general, las contracciones se evitan cuando se usan en la escritura formal, pero de manera informal, se usan bastante. Existen varias formas de contracciones que están ligadas al tipo de verbos auxiliares que nos dan contracciones normales, contracciones negadas y otras contracciones coloquiales especiales, algunas de las cuales pueden no tener auxiliares.
 
Recuerde, sin embargo, que algunas de las contracciones pueden tener múltiples formas. Para simplificar, he tomado una de estas formas expandidas para cada contracción. El siguiente paso, para expandir las contracciones, usa el siguiente fragmento de código

In [19]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [20]:
def expand_contractions(sentence, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
    expanded_sentence = contractions_pattern.sub(expand_match, sentence)
    return expanded_sentence

El codigo anterior usa función `expand_match` dentro de la función principal `expand_contractions` para encontrar cada contracción que coincida con el patrón de expresiones regulares que creamos a partir de todas las contracciones en nuestro diccionario `CONTRACTION_MAP`. Al emparejar cualquier contracción, la sustituimos con su correspondiente versión expandida y conservamos el caso correcto de la palabra

In [21]:
expanded_corpus = [expand_contractions(sentence, CONTRACTION_MAP) for sentence in cleaned_corpus]
print(expanded_corpus)

['The brown fox was not that quick and he could not win the race', 'Hey that is a great deal! I just bought a phone for 199', 'You will learn a lot in the book. Python is an amazing language !']


## 4. Case Conversion

A menudo queremos modificar las mayúsculas y minúsculas de las palabras o las oraciones para facilitar las cosas, como hacer coincidir palabras o tokens específicos. Por lo general, hay dos tipos de operaciones de conversión de casos que se usan mucho. Estas son conversiones de minúsculas y mayúsculas, donde un cuerpo de texto se convierte completamente a minúsculas o mayúsculas. También hay otras formas, como el caso de la oración o el caso propio. Minúsculas es una forma donde todas las letras del texto son minúsculas, y en mayúsculas todas están en mayúscula.

In [22]:
print(corpus[0].lower())
print(corpus[0].upper())

the brown fox wasn't that quick and he couldn't win the race
THE BROWN FOX WASN'T THAT QUICK AND HE COULDN'T WIN THE RACE


## 5. Remover Stopwords

Las palabras vacías, a veces palabras vacías escritas, son palabras que tienen poco o ningún significado. Por lo general, se eliminan del texto durante el procesamiento para conservar las palabras que tienen el máximo significado y contexto. Las palabras vacías suelen ser palabras que terminan apareciendo con mayor frecuencia si agrega cualquier corpus de texto basado en tokens singulares y verifica sus frecuencias. Palabras como a, the , me , etc. son palabras vacías. No existe una lista universal o exhaustiva de palabras vacías. Cada dominio o idioma puede tener su propio conjunto de palabras vacías

In [23]:
def remove_stopwords(tokens):
    stopword_list = nltk.corpus.stopwords.words('english')
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    return filtered_tokens

Aprovechamos el uso de nltk , que tiene una lista de palabras vacías para inglés filtramos todos los tokens que correspondan a palabras vacías. Usamos nuestra función `tokenize_text` para tokenizar el cuerpo expandido que obtuvimos en la sección anterior y luego eliminamos las palabras vacías necesarias usando la función anterior.

In [24]:
expanded_corpus_tokens = [tokenize_text(text) for text in expanded_corpus]
expanded_corpus_tokens

[[['The',
   'brown',
   'fox',
   'was',
   'not',
   'that',
   'quick',
   'and',
   'he',
   'could',
   'not',
   'win',
   'the',
   'race']],
 [['Hey', 'that', 'is', 'a', 'great', 'deal', '!'],
  ['I', 'just', 'bought', 'a', 'phone', 'for', '199']],
 [['You', 'will', 'learn', 'a', 'lot', 'in', 'the', 'book', '.'],
  ['Python', 'is', 'an', 'amazing', 'language', '!']]]

In [25]:
filtered_list_3 = [[remove_stopwords(tokens) for tokens in sentence_tokens] \
                   for sentence_tokens in expanded_corpus_tokens]
filtered_list_3

[[['The', 'brown', 'fox', 'quick', 'could', 'win', 'race']],
 [['Hey', 'great', 'deal', '!'], ['I', 'bought', 'phone', '199']],
 [['You', 'learn', 'lot', 'book', '.'],
  ['Python', 'amazing', 'language', '!']]]

## 6. Correccion de palabras

Texto incorrecto puede ser palabras que tienen errores de ortografía, así como palabras con varias letras repetidas que no contribuyen mucho a su significado general. Para ilustrar algunos ejemplos, la palabra **finally^^ podría escribirse erróneamente como **fianlly**, o alguien que exprese una emoción intensa podría escribirla como **finalllllyyyyyy**.

### 6.1 Correccion de caracteres repetidos
El primer paso en nuestro algoritmo sería identificar los caracteres repetidos en una palabra usando un patrón de expresiones regulares y luego usar una sustitución para eliminar los caracteres uno por uno.

Considera la palabra **finallyyy** del ejemplo anterior. El patrón `r'(\w*)(\w)\2(\w*)'` puede usarse para identificar caracteres que ocurren dos veces entre otros caracteres de la palabra, y en cada paso intentaremos eliminar uno de los repetidos usando una sustitución para la coincidencia utilizando los grupos de coincidencia de expresiones regulares (grupos 1, 2 y 3) usando el patrón `r'\1\2\3'` y luego se sigue iterando a través de este proceso hasta que no queden caracteres repetidos.

In [27]:
old_word = 'finalllyyy'
repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
match_substitution = r'\1\2\3'
step = 1
while True:
    # remove one repeated character
    new_word = repeat_pattern.sub(match_substitution,old_word)
    if new_word != old_word:
        print('Paso: {} Palabra: {}'.format(step, new_word))
        step += 1 # update step 
        # update old word to last substituted state
        old_word = new_word
        continue
    else:
        print("Final word:", new_word)
        break

Paso: 1 Palabra: finalllyy
Paso: 2 Palabra: finallly
Paso: 3 Palabra: finally
Paso: 4 Palabra: finaly
Final word: finaly


El fragmento anterior muestra cómo se elimina un carácter repetido en cada etapa hasta que terminamos con la palabra finalmente al *finaly*. Sin embargo, semánticamente, esta palabra es incorrecta: la palabra correcta fue **finally**, que obtuvimos en el paso 3. Ahora utilizaremos el corpus de WordNet para verificar las palabras válidas en cada etapa y terminar el ciclo una vez se obtiene la opcion correcta. Esto introduce la corrección semántica necesaria para nuestro algoritmo

In [28]:
from nltk.corpus import wordnet
old_word = 'finalllyyy'
repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
match_substitution = r'\1\2\3'
step = 1
while True:
    # chequear la semantica 
    if wordnet.synsets(old_word):
        print("Palabra final corregida:", old_word)
        break
    # remover  caracteres repetidos
    new_word = repeat_pattern.sub(match_substitution,old_word)
    if new_word != old_word:
        print('Paso: {} Palabras: {}'.format(step, new_word))
        step += 1 # update 
        # update la old word por su estado cambiado
        old_word = new_word
        continue
    else:
        print("Palabra final:", new_word)
        break

Paso: 1 Palabras: finalllyy
Paso: 2 Palabras: finallly
Paso: 3 Palabras: finally
Palabra final corregida: finally


Ahora si lo hacemos mas generico tenemos:

In [29]:
from nltk.corpus import wordnet
def remove_repeated_characters(tokens):
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    match_substitution = r'\1\2\3'
    def replace(old_word):
        if wordnet.synsets(old_word):
            return old_word
        new_word = repeat_pattern.sub(match_substitution, old_word)
        return replace(new_word) if new_word != old_word else new_word
    correct_tokens = [replace(word) for word in tokens]
    return correct_tokens

Ese fragmento usa la función interna `replace()` para emular básicamente el comportamiento de nuestro algoritmo, ilustrado anteriormente, y luego llamarlo repetidamente en cada token en una oración en la función externa `remove_repeated_characters()`.

In [30]:
sample_sentence = 'My schooool is realllllyyy amaaazingggg'
sample_sentence_tokens = tokenize_text(sample_sentence)[0]
print(sample_sentence_tokens)

['My', 'schooool', 'is', 'realllllyyy', 'amaaazingggg']


In [31]:
print(remove_repeated_characters(sample_sentence_tokens))

['My', 'school', 'is', 'really', 'amazing']


PErfecto hasta acá hemos realizado varias tareas como **Tokenizar, Remover Caracteres Especiales, Expandir Contracciones, Convertir a minusculas, remocion de stopwords así como remover caracteres repetidos en palabras**. En el proximo tutorial veremos como terminar de hacer la correccion ortografica de nuestro texto y aprenderas sobre los procesos de Stemming y Lemantizacion